# Load Generator Notebook

In [8]:
GATEWAY_URL = '34.133.80.64:80'

def generate_load(num_clients, num_reqs):
    cmd = "docker run load-test -h {url} -r {num_reqs} -c {num_clients}".format(url = GATEWAY_URL, num_reqs = num_reqs, num_clients = num_clients)
    return cmd

def generate_load_args(num_clients, num_reqs):
    args = "-h {url} -r {num_reqs} -c {num_clients}".format(url = GATEWAY_URL, num_reqs = num_reqs, num_clients = num_clients)
    return args

## First Method: Use Docker Client to run load generator

In [9]:
import docker
client = docker.from_env()
args = generate_load_args(250, 2500)
client.containers.run("load-test", args)

KeyboardInterrupt
2021-12-17T20:10:00Z


KeyboardInterrupt: 

## Second Method: Use `os.System` to run load generator

In [14]:
import os
cmd = generate_load(1, 100)
#result = os.system(cmd)
# import commands
# status, output = commands.getstatusoutput(cmd)

In [24]:
import subprocess
output = subprocess.getoutput(cmd);
print(output)

Locust file: /config/locustfile.py
34.133.80.64:80 is not accessible


In [19]:
output

'Locust file: /config/locustfile.py\nWill run /config/locustfile.py against 34.133.80.64:80. Spawning 1 clients and 100 total requests.\n[2021-12-17 20:12:31,763] d56cf52c3347/INFO/locust.main: Starting Locust 0.7.5\n[2021-12-17 20:12:31,764] d56cf52c3347/INFO/locust.runners: Hatching and swarming 1 clients at the rate 5 clients/s...\n[2021-12-17 20:12:31,965] d56cf52c3347/INFO/locust.runners: All locusts hatched: Web: 1\n[2021-12-17 20:12:31,965] d56cf52c3347/INFO/locust.runners: Resetting stats\n\n[2021-12-17 20:12:37,904] d56cf52c3347/INFO/locust.runners: All locusts dead\n\n[2021-12-17 20:12:37,905] d56cf52c3347/INFO/locust.main: Shutting down (exit code 1), bye.\n Name                                                          # reqs      # fails     Avg     Min     Max  |  Median   req/s\n--------------------------------------------------------------------------------------------------------------------------------------------\n GET /                                                

In [6]:
import re
def parse(result):
    """
    input: os system result
    output: two dictionary of data
    
    """
    delimiter = "--------------------------------------------------------------------------------------------------------------------------------------------"
    data = output.split(delimiter)

    # create dictionary of request data
    request_data = data[1]
    request_data_key, request_data_val = extract(request_data, 7)
    request_data_dict = dict(zip(request_data_key, request_data_val))

    # create dictionary of request data
    request_complete_percent = data[3]
    request_complete_percent_key, request_complete_percent_val = extract(request_complete_percent, 10)
    request_complete_percent_dict = dict(zip(request_complete_percent_key, request_complete_percent_val))    
    return request_data_dict, request_complete_percent_dict


def extract(data, num_col):
    # select all values
    regex_val = "([0-9]+\([0-9]+\.[0-9]+\%\)|[0-9]+\.[0-9]+| [0-9]+ )"
    val = re.findall(regex_val, data)


    # remove space
    val = [x.strip(' ') for x in val]

    # group each row into nested list
    val = [val[i:i + num_col] for i in range(0, len(val), num_col)]
    
    # select all keys
    regex_key = "[A-Z]+ [^\s]+"
    key = re.findall(regex_key, data)
    return key, val


In [7]:
request_data_dict, request_complete_percent_dict = parse(output)

In [ ]:
request_data_dict

## Third Method: Directly use Locus within Notebook

In [ ]:
from locust.env import Environment